# Report on Adversarial Examples
This notebook should 
1. Train new models with different parameters
1. for each model, generate both a **targeted** and **untargeted** PGD attack
1. Plot the results

In [ ]:
import subprocess
from pathlib import Path 
from matplotlib import pyplot as plt 
import json 

prefix = 'experiments/'
adv_dir = 'attacks/'

## Train New Models
Each with a specific set of parameters, including data parameters

In [ ]:
def make_args(param):
    """
    turn parameter dictionary into list of arguments
    """
    out = []
    for k, v in param.items():
        out.append('--' + k)
        out.append(str(v))
    return out 

durations = [50, 100, 150]
whichs = [1, 2, 3, 4]
model_params = {
    f'{prefix}{which=}_{duration=}': {'duration': duration, 'which': which} 
    for duration in durations for which in whichs
}


# create models
for name, v in model_params.items():
    subprocess.run(['python', 'lstm.py'] + make_args(v) + ['--target_directory', name])

## Generate Adversarial Examples
Save the results with the model in the appropriate folder, both for targeted and untargeted attacks. 

Parameters of the attack:
- Attack type (targeted or untargeted)
- Perturbation size

In [ ]:
# for each model, make a number of attacks 
# To that end, make a subfolder "attacks" for each of the models
for name in model_params.keys():
    attack_types = ['targeted', 'untargeted']
    perturbation_sizes = [.01, .05, .1]

    attack_params = {
        f'{adv_dir}{attack_type=}_{perturbation_size=}/': {'attack_type': attack_type, 'perturbation_size': perturbation_size} 
        for attack_type in attack_types for perturbation_size in perturbation_sizes
    }
    for attack_name, v in attack_params.items():
        subprocess.run(['python', 'attack.py'] + make_args(v) + ['--source_directory', name, '--target_directory', attack_name])

## Plot the Results

In [ ]:
# For each model, plot the accuracy against perturbation size, both targeted and untargeted
cwd = Path().cwd()
model_name = 'max'
for experiment in model_params.keys():
    # get accuracy and adv accuracy
    dir = cwd / experiment
    if model_name == 'max':
        model_name = max([str(i) for i in dir.iterdir()])
    dir = dir / model_name
    with open(str(dir / 'stats.json')) as f:
        stats = json.load(f)
    acc = stats['acc']
    attack_dir = dir / 'attacks'
    adv_accs = {'targeted': [], 'untargeted': []}
    for attack in attack_dir.iterdir():
        with open(str(attack / 'adv_stats.json')) as f:
            adv_stats = json.load(f)
        adv_acc = adv_stats['acc']
        if 'untargeted' in str(attack):
            adv_accs['untargeted'].append(adv_acc)
        else:
            adv_accs['targeted'].append(adv_acc)
    
    # make the plot
    fig, ax = plt.subplots(1, 1)
    ax.plot(perturbation_sizes, [acc] * len(perturbation_sizes), label='regular')
    ax.plot(perturbation_sizes, adv_accs['targeted'], label='targeted')
    ax.plot(perturbation_sizes, adv_accs['untargeted'], label='untargeted')
    ax.legend()
    ax.set_title(experiment)
    fig.tight_layout()
    fig.savefig(str(dir / 'accuracies.png'))